In [33]:
import tensorflow as tf
import cv2
import pandas as pd
import numpy as np
import seaborn as sns
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
import os

In [12]:
from keras.preprocessing.image import ImageDataGenerator

In [26]:
img_width, img_height = 256, 256
batch_size = 32


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    validation_split=0.15
)


train_path = r"C:\Users\Siddhartha Devan V\Downloads\datasets\Tomato disease\train"

In [29]:
train_generator = train_datagen.flow_from_directory(
    train_path,  # Path to the parent directory containing class folders
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  
    subset='training',
    classes = {'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 
               'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 
               'Tomato_mosaic_virus': 8,'powdery_mildew': 9, 'healthy': 10 }
)

Found 21976 images belonging to 11 classes.


In [30]:
valid_generator = train_datagen.flow_from_directory(
    train_path,  # Path to the parent directory containing class folders
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  
    subset='validation',
    classes = {'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 
               'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 
               'Tomato_mosaic_virus': 8,'powdery_mildew': 9, 'healthy': 10 }
)

Found 3872 images belonging to 11 classes.


In [19]:
print(train_generator.class_indices)

{'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato_mosaic_virus': 8, 'powdery_mildew': 9, 'healthy': 10}


In [23]:
test_datagen = ImageDataGenerator(
    rescale=1. / 255
)


test_path = r"C:\Users\Siddhartha Devan V\Downloads\datasets\Tomato disease\valid"

In [24]:
test_generator = test_datagen.flow_from_directory(
    test_path,  # Path to the parent directory containing class folders
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',  # Assuming you have two classes
    classes = {'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 
               'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 
               'Tomato_mosaic_virus': 8,'powdery_mildew': 9, 'healthy': 10 }
)

Found 6684 images belonging to 11 classes.


In [31]:
print(test_generator.class_indices)

{'Bacterial_spot': 0, 'Early_blight': 1, 'Late_blight': 2, 'Leaf_Mold': 3, 'Septoria_leaf_spot': 4, 'Spider_mites Two-spotted_spider_mite': 5, 'Target_Spot': 6, 'Tomato_Yellow_Leaf_Curl_Virus': 7, 'Tomato_mosaic_virus': 8, 'powdery_mildew': 9, 'healthy': 10}


In [32]:
from architecture_module import QuadDense, ResidualBlock, InceptionBlock

In [ ]:
class InceptionResidualNetwork(Model):
    def __init__(self):
        self.conv1 = tf.keras.layers.Conv2D(kernel_size=(3,3), filters = 32)
        self.activation = tf.keras.layers.Activation('relu')
        self.pool1 = tf.keras.layers.MaxPooling2D(pool_size=(3,3))
        self.bn1 = tf.keras.layers.BatchNormalization()

        self.conv2 = tf.keras.layers.Conv2D(kernel_size=(5,5), filters=64)
        self.pool2 = tf.keras.layers.MaxPooling2D(pool_size=(3,3))
        self.bn2 = tf.keras.layers.BatchNormalization()

        self.inception_block_1 = InceptionBlock(n_filters=64)

        self.conv3 = tf.keras.layers.Conv2D(kernel_size=(3,3), filters = 128)
        self.pool3 = tf.keras.layers.MaxPooling2D(pool_size=(3,3))
        self.bn3 = tf.keras.layers.BatchNormalization()

        self.residual_1 = ResidualBlock(n_filters=64, kernel_size=(3,3))

        self.flatten = tf.keras.layers.Flatten()
        
        self.quad_dense1 = QuadDense(units=500, activation = 'relu')
        self.dense2 = tf.keras.layers.Dense(100, activation='relu')
        self.dense3 = tf.keras.layers.Dense(11, activation='softmax')

    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.activation(x)
        x = self.bn1(x)
        x = self.pool1(x)

        x = self.conv2(x)
        x = self.activation(x)
        x = self.bn2(x)
        x = self.pool2(x)

        x = 



